In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
import nltk
import string
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from tqdm import tqdm

In [2]:
comments=pd.read_csv('./final.csv')

In [3]:
comments.shape

(346959, 1)

In [4]:
comments.isnull().sum()

comments    0
dtype: int64

In [5]:
comments.head()

,comments
0,Stephanie was a wonderful host! Her apartment ...
1,Such a wonderful place and very close to the m...
2,I just got back from a trip to NYC during whic...
3,Stephanie's offered all the most important thi...
4,"Stephanie was really nice, ftiendly and helpfu..."


In [6]:
comments.dropna(inplace=True)

In [7]:
comments.isnull().sum()

comments    0
dtype: int64

In [8]:
import re
import numpy as np

In [9]:
import gensim

In [10]:
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [11]:
import spacy

In [12]:
# for plotting
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt

In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>


False

In [14]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [15]:
# Convert email body to list
data = comments.comments.values.tolist()

In [16]:
# tokenize - break down each sentence into a list of words
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [17]:
data_words=list(sent_to_words(data))

In [18]:
print(data_words[3])

['stephanie', 'offered', 'all', 'the', 'most', 'important', 'things', 'warm', 'welcome', 'into', 'comfortable', 'home', 'comfortable', 'bed', 'in', 'quiet', 'room', 'fresh', 'clean', 'towels', 'blankets', 'and', 'easy', 'access', 'to', 'manhattan', 'finding', 'myself', 'travelling', 'to', 'nyc', 'in', 'the', 'future', 'feel', 'already', 'have', 'open', 'invitation', 'to', 'make', 'home', 'away', 'from', 'home', 'through', 'stephanie', 'generousity']


In [19]:
from gensim.models.phrases import Phrases, Phraser

In [20]:
# Build the bigram and trigram models
bigram = Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = Phrases(bigram[data_words], threshold=100)

In [21]:
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)

In [22]:
print(trigram_mod[bigram_mod[data_words[35]]])

['best', 'host', 'ever', 'had', 'stefanie', 'and', 'her', 'family', 'are', 'very', 'kind', 'helpful', 'and', 'accommodating', 'you', 'will', 'feel', 'at', 'home', 'here', 'immediately', 'the', 'place', 'is', 'very', 'nice', 'clean', 'and', 'you', 'can', 'beat', 'the', 'access', 'to', 'both', 'the', 'and', 'as', 'well', 'as', 'the', 'and', 'subway', 'lines', 'both', 'each', 'blocks', 'away', 'will', 'definitely', 'come', 'back']


In [23]:
#remove stop_words, make bigrams and lemmatize
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [24]:
data_words_nostops = remove_stopwords(data_words)

In [25]:
data_words_bigrams = make_bigrams(data_words_nostops)

In [26]:
import en_core_web_sm

In [27]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = en_core_web_sm.load(disable=['parser', 'ner'])

In [28]:
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [29]:
import pickle

In [31]:
f=open("./model/data_lemmatized_final.pkl",'wb')
pickle.dump(data_lemmatized,f)
f.close()

In [ ]:
#data_lemmatized = []
#with (open("data_lemmatized.pkl", "rb")) as openfile:
    #while True:
        #try:
        #    data_lemmatized.append(pickle.load(openfile))
        #except EOFError:
        #    break

In [17]:
#data_lemmatized=data_lemmatized[0]

In [32]:
len(data_lemmatized)

346959

In [33]:
#create a dictionary and corpus for (LDA topic modeling)

#creating the dictionary

idtoword=corpora.Dictionary(data_lemmatized)
#create corpus

texts=data_lemmatized

corpus=[idtoword.doc2bow(text) for text in texts]

In [34]:
len(corpus)

346959

# Gensim LDA Model -Topic Modeling

In [42]:
#build our LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=idtoword,
                                           num_topics=5,
                                           workers=3,
                                           random_state=100,
                                           eval_every=1,
                                           chunksize=100,
                                           passes=10)


In [43]:
f=open("./model/lda_model_final.pkl",'wb')
pickle.dump(lda_model,f)
f.close()

In [89]:
#pickle.dump(lda_model, open("./model/lda.pickle", "wb"))
pickle.dump(idtoword, open("./model/idatoword_final.pickle", "wb"))
pickle.dump(corpus, open("./model/corpus_final.pickle", "wb"))


In [44]:
print(lda_model.print_topics())

[(0, '0.041*"close" + 0.036*"walk" + 0.032*"subway" + 0.031*"apartment" + 0.028*"restaurant" + 0.028*"great" + 0.023*"place" + 0.019*"minute" + 0.018*"station" + 0.018*"location"'), (1, '0.022*"host" + 0.020*"day" + 0.013*"night" + 0.013*"arrival" + 0.012*"check" + 0.012*"apartment" + 0.012*"get" + 0.010*"reservation" + 0.010*"cancel" + 0.010*"place"'), (2, '0.044*"stay" + 0.029*"place" + 0.022*"make" + 0.019*"feel" + 0.018*"home" + 0.015*"time" + 0.015*"thank" + 0.014*"back" + 0.014*"come" + 0.014*"host"'), (3, '0.074*"room" + 0.037*"good" + 0.030*"clean" + 0.029*"bed" + 0.024*"apartment" + 0.024*"bathroom" + 0.023*"nice" + 0.018*"well" + 0.018*"comfortable" + 0.016*"kitchen"'), (4, '0.097*"great" + 0.063*"place" + 0.055*"stay" + 0.054*"location" + 0.041*"host" + 0.037*"clean" + 0.029*"recommend" + 0.027*"apartment" + 0.021*"nice" + 0.018*"good"')]


In [45]:
lda_model.save('ldafinal.model')

In [46]:
# Visualize the topics
pyLDAvis.enable_notebook(sort=True)
vis = pyLDAvis.gensim.prepare(lda_model, corpus, idtoword)
pyLDAvis.display(vis)

/Users/ikhyvicky/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [47]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))



Perplexity:  -6.575325572154794


In [48]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=idtoword, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

/Users/ikhyvicky/anaconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:504: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/Users/ikhyvicky/anaconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:506: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):
/Users/ikhyvicky/anaconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:320: DeprecationWarning: Numpy has detected that you (may be) writing to an array with
overlapping memory from np.broadcast_arrays. If this is intentional
set the WRITEABLE flag True or make a copy immediately before writing.
  i, j, x)
/Users/ikhyvicky/anaconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:320: DeprecationWarning: Numpy ha


Coherence Score:  0.5694115997238247


In [49]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [50]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data)

In [51]:
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

In [52]:
df_dominant_topic.shape

(346959, 5)

In [53]:
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,4.0,0.6866,"great, place, stay, location, host, clean, rec...",Stephanie was a wonderful host! Her apartment ...
1,1,2.0,0.4246,"stay, place, make, feel, home, time, thank, ba...",Such a wonderful place and very close to the m...
2,2,2.0,0.4472,"stay, place, make, feel, home, time, thank, ba...",I just got back from a trip to NYC during whic...
3,3,2.0,0.5302,"stay, place, make, feel, home, time, thank, ba...",Stephanie's offered all the most important thi...
4,4,4.0,0.4902,"great, place, stay, location, host, clean, rec...","Stephanie was really nice, ftiendly and helpfu..."
5,5,2.0,0.6852,"stay, place, make, feel, home, time, thank, ba...",I was pleasantly surprised with my whole stay ...
6,6,0.0,0.5188,"close, walk, subway, apartment, restaurant, gr...",Stephanie's apt was great. Very convenient to ...
7,7,2.0,0.3343,"stay, place, make, feel, home, time, thank, ba...",my husband and i had such a great time staying...
8,8,2.0,0.4982,"stay, place, make, feel, home, time, thank, ba...","When I first arrived at Stephanie's, she was v..."
9,9,2.0,0.5963,"stay, place, make, feel, home, time, thank, ba...",awesome couldn't have been better.


In [54]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [55]:
sia = SentimentIntensityAnalyzer()

In [56]:
example=df_dominant_topic.loc[:10,:]

In [57]:
example.shape

(11, 5)

In [58]:
for sentence in example.Text:
    # Assess sentiment.
    sentiment = sia.polarity_scores(sentence)["compound"]
    print(sentence)
    print(sentiment)
    print('\n')

Stephanie was a wonderful host! Her apartment was beautiful and extremely clean. We had a very nice (and affordable!) stay in the city with private room and own bathroom.
0.9357


Such a wonderful place and very close to the metro station on the line 1 2 3. Your She was awesome with us, It was a pleasure to stay in her appt. she's french, so last word ...
Stephanie thank you again for your hospitality. We had a great time with you. Very available for us with lots of advice, I would particularly recommend this address which is more for a reasonable price. The subway is just 1 block from the apartment and the 1-2-3 line is the central and most convenient NYC. her neighborhood is really very nice and more !!! Go there with closed eyes.
0.9727


I just got back from a trip to NYC during which I stayed at Stephanie's appartment. The bedroom and private bathroom that comes with it were both kept very clean and are spacious so that I could unpack my stuff and put my clothes away in the closet

In [59]:
sentiment_scores=[]
for sentence in df_dominant_topic.Text:
    sentiment = sia.polarity_scores(sentence)["compound"]
    sentiment_scores.append(sentiment)
    

In [60]:
df_dominant_topic['sentiment_score'] = pd.Series(sentiment_scores, index=df_dominant_topic.index)

In [61]:
df_dominant_topic.shape

(346959, 6)

In [62]:
df_dominant_topic.head()

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,sentiment_score
0,0,4.0,0.6866,"great, place, stay, location, host, clean, rec...",Stephanie was a wonderful host! Her apartment ...,0.9357
1,1,2.0,0.4246,"stay, place, make, feel, home, time, thank, ba...",Such a wonderful place and very close to the m...,0.9727
2,2,2.0,0.4472,"stay, place, make, feel, home, time, thank, ba...",I just got back from a trip to NYC during whic...,0.9620
3,3,2.0,0.5302,"stay, place, make, feel, home, time, thank, ba...",Stephanie's offered all the most important thi...,0.9626
4,4,4.0,0.4902,"great, place, stay, location, host, clean, rec...","Stephanie was really nice, ftiendly and helpfu...",0.9335


In [63]:
#to get a sense of what negative comments will look like
negative_sentiments=df_dominant_topic[df_dominant_topic['sentiment_score']<0]

In [64]:
negative_sentiments.reset_index(drop=True,inplace=True)

In [65]:
negative_sentiments.shape

(8996, 6)

In [66]:
negative_sentiments.sort_values('sentiment_score').head(20)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,sentiment_score
3565,162817,1.0,0.7557,"host, day, night, arrival, check, apartment, g...",This is a filthy hole in a filthy old building...,-0.9971
2277,111046,1.0,0.7482,"host, day, night, arrival, check, apartment, g...",Dangerous host \nI really hate it when I write...,-0.9966
8788,338855,1.0,0.7021,"host, day, night, arrival, check, apartment, g...",We had a very bad experience with this Airbnb ...,-0.9965
3748,169226,1.0,0.8036,"host, day, night, arrival, check, apartment, g...",Here is a list of issues to consider. We reach...,-0.9963
7510,297779,1.0,0.7635,"host, day, night, arrival, check, apartment, g...",I really do not enjoy having to write bad revi...,-0.9963
6872,277427,1.0,0.8042,"host, day, night, arrival, check, apartment, g...",From the moment me and my sister booked this A...,-0.9956
1572,82528,1.0,0.7815,"host, day, night, arrival, check, apartment, g...",BAD BAD TERRIEBLE. FROM THE BEGINNING. Very ru...,-0.9949
212,12044,1.0,0.7388,"host, day, night, arrival, check, apartment, g...","During the stay, Zooey and I felt very happy t...",-0.9948
1024,57391,1.0,0.7901,"host, day, night, arrival, check, apartment, g...",My wife and 4 kids had a very dissatisfying st...,-0.9947
2526,124547,1.0,0.6990,"host, day, night, arrival, check, apartment, g...",We have used AirBnb and (Hidden by Airbnb) as ...,-0.9946


In [67]:
negative_sentiments.Text.iloc[8788]

"We had a very bad experience with this Airbnb stay. Zheng was very friendly as a person and tried to help, but there were so many things wrong and illegal about her setup, that it led to the worst Airbnb experience we have ever had. Zheng has put her rental apartment on Airbnb without permission from her leasing office (as she told us herself). Hence, she lied about many things in her listing and to us when we booked. She gave us a wrong  address since she didn't want us to reach her main entrance/lobby or use the main elevator since she thought her leasing agency folks will catch her doing Airbnb illegally.  Further, she asked us not to use the lobby, main elevator and the correct address (anybody for deliveries would then reach the lobby and meet the leasing office people). Instead, she asked us to use a freight elevator on one side of the building to enter the property. This led to a series of bad experiences for us. I travel for business and I book using Airbnb for business. One o

In [68]:
negative_sentiments.Dominant_Topic.nunique() #this just means people  can talk negative things for each topic

5

In [69]:
#putting them back together
df=pd.read_csv('./removed_january.csv')

In [70]:
df.dropna(inplace=True)

In [71]:
df.shape

(346959, 6)

In [72]:
df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2515,198,2008-10-13,2603,Jenny,Stephanie was a wonderful host! Her apartment ...
1,2515,859,2009-03-08,8455,Roland,Such a wonderful place and very close to the m...
2,2515,1083,2009-03-25,9759,Cem,I just got back from a trip to NYC during whic...
3,2515,1107,2009-03-27,9193,Holly,Stephanie's offered all the most important thi...
4,2515,2175,2009-05-09,7048,Alessandra,"Stephanie was really nice, ftiendly and helpfu..."


In [73]:
df_dominant_topic.shape

(346959, 6)

In [74]:
df_dominant_topic.head()

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,sentiment_score
0,0,4.0,0.6866,"great, place, stay, location, host, clean, rec...",Stephanie was a wonderful host! Her apartment ...,0.9357
1,1,2.0,0.4246,"stay, place, make, feel, home, time, thank, ba...",Such a wonderful place and very close to the m...,0.9727
2,2,2.0,0.4472,"stay, place, make, feel, home, time, thank, ba...",I just got back from a trip to NYC during whic...,0.9620
3,3,2.0,0.5302,"stay, place, make, feel, home, time, thank, ba...",Stephanie's offered all the most important thi...,0.9626
4,4,4.0,0.4902,"great, place, stay, location, host, clean, rec...","Stephanie was really nice, ftiendly and helpfu...",0.9335


In [75]:
#drop original non-translated comments
df.drop(columns=['comments'],axis=1,inplace=True)

In [76]:
df.reset_index(inplace=True)

In [77]:
df_dominant_topic.drop(columns=['Document_No'],axis=1,inplace=True)

In [78]:
#combine the two together 
frames=[df,df_dominant_topic]
finaldf=pd.concat(frames,axis=1)

In [79]:
finaldf.head()

,index,listing_id,id,date,reviewer_id,reviewer_name,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,sentiment_score
0,0,2515,198,2008-10-13,2603,Jenny,4.0,0.6866,"great, place, stay, location, host, clean, rec...",Stephanie was a wonderful host! Her apartment ...,0.9357
1,1,2515,859,2009-03-08,8455,Roland,2.0,0.4246,"stay, place, make, feel, home, time, thank, ba...",Such a wonderful place and very close to the m...,0.9727
2,2,2515,1083,2009-03-25,9759,Cem,2.0,0.4472,"stay, place, make, feel, home, time, thank, ba...",I just got back from a trip to NYC during whic...,0.9620
3,3,2515,1107,2009-03-27,9193,Holly,2.0,0.5302,"stay, place, make, feel, home, time, thank, ba...",Stephanie's offered all the most important thi...,0.9626
4,4,2515,2175,2009-05-09,7048,Alessandra,4.0,0.4902,"great, place, stay, location, host, clean, rec...","Stephanie was really nice, ftiendly and helpfu...",0.9335


In [80]:
finaldf.shape

(346959, 11)

In [81]:
finaldf.to_csv('./finaldf.csv',index=False)

In [82]:
#now the tedious part is to link listing id to negative sentiment scores
final_negative_sentiments=finaldf[finaldf['sentiment_score']<0]

In [83]:
final_negative_sentiments.shape  #8996 comments with negative comments

(8996, 11)

In [84]:
final_negative_sentiments.listing_id.nunique() #3840 unique listings out of 13322 listings are negatively commented  29% of the listings

3840

In [85]:
df.listing_id.nunique() 

13322

In [86]:
sorted(final_negative_sentiments['listing_id'].value_counts(),reverse=True)

[27,
 25,
 22,
 21,
 21,
 21,
 20,
 19,
 18,
 17,
 17,
 17,
 17,
 17,
 17,
 16,
 16,
 15,
 14,
 14,
 14,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 6,
 6,
 6,
 6,
 6,
 6,
 6

In [87]:
listings_sentiments=finaldf.groupby(['listing_id','Dominant_Topic']).agg({'index':'count', 'sentiment_score':'mean'})

In [88]:
listings_sentiments.head(20)

index  sentiment_score
listing_id Dominant_Topic                        
2515       0.0                29         0.773814
           1.0                12         0.346767
           2.0                85         0.867938
           3.0                22         0.851195
           4.0                27         0.882193
2539       0.0                 4         0.584750
           2.0                 2         0.943450
           3.0                 2         0.699100
           4.0                 1         0.624900
3330       0.0                 4         0.924525
           1.0                 3         0.323033
           2.0                16         0.964581
           3.0                 4         0.981600
           4.0                12         0.911567
5022       0.0                 4         0.725500
           2.0                 1         0.990300
           4.0                 4         0.889125
5216       0.0                 5         0.729980
           1.0                 9         0.707267
           2.0                 6         0.880950

In [90]:
listings_sentiments.to_csv('./listings_sentiments.csv')